In [ ]:
import pandas as pd
import datetime
from math import ceil
import matplotlib.pyplot as plt
import torch
import numpy as np
from importlib import reload
import random
from random import shuffle
from dateutil.parser import parse

import rl4pm_lib.preprocessing as preprocessing
reload(preprocessing)
from IPython import display

In [ ]:
test_df = pd.read_csv('datasets/test_df_nr.csv')
test_df['timestamp'] = test_df['timestamp'].apply(lambda x: parse(x))

train_df = pd.read_csv('datasets/train_df_nr.csv')
train_df['timestamp'] = train_df['timestamp'].apply(lambda x: parse(x))

train_df['activity'] = train_df['activity'].apply(lambda x: str(x))
test_df['activity'] = test_df['activity'].apply(lambda x: str(x))

Create val dataset

In [ ]:
from random import sample
train_idx = set(train_df['trace_id'].values)
val_idx = sample(train_idx, int(0.25 * len(train_idx)))

train_idx_ = []
for i in train_idx:
    if i not in val_idx:
        train_idx_.append(i)
train_idx = train_idx_

val_df = train_df[train_df['trace_id'].isin(val_idx)]
train_df = train_df[train_df['trace_id'].isin(train_idx)]

train_df.reset_index(inplace=True, drop=True)
val_df.reset_index(inplace=True, drop=True)

In [ ]:
train_df

### Propro like for SklEarn

In [ ]:
import rl4pm_lib.preprocessing as preprocessing

column_feature = {'tt': 0, 'te': 1, 'tw': 2}
prepro = preprocessing.DfPreprocesser()
prepro.fit(train_df)
train_df_pr = prepro.transform(train_df)
test_df_pr = prepro.transform(test_df)
val_df_pr = prepro.transform(val_df)

In [ ]:
print(f'test_df_pr nans = {test_df_pr.isna().sum().sum()}')
print(f'train_df_pr nans = {train_df_pr.isna().sum().sum()}')
print(f'val_df_pr nans = {val_df_pr.isna().sum().sum()}')

In [ ]:
train_df_pr.rename(columns={i+1: f'activ_{str(i+1)}' for i in range(6)}, inplace=True)
test_df_pr.rename(columns={i+1: f'activ_{str(i+1)}' for i in range(6)}, inplace=True)
val_df_pr.rename(columns={i+1: f'activ_{str(i+1)}' for i in range(6)}, inplace=True)

In [ ]:
scaler = preprocessing.PaperScalerPd(column_feature, drop_useless=False)
scaler.fit(train_df_pr)
train_df_pr_sc = scaler.transform(train_df_pr)
test_df_pr_sc = scaler.transform(test_df_pr)
val_df_pr_sc = scaler.transform(val_df_pr)

## Ok let's create data loader & dataset

In [ ]:
from rl4pm_lib.lstm_supervised import ProcessesDataset

In [ ]:
win_len = 2
batch_size = 1024
n_lstm = 1
m_lstm = 2
hidden_layer = 128
n_epoch = 100
n_classes = len(set(train_df['activity'].values))

In [ ]:
test_ds = ProcessesDataset(test_df_pr_sc, win_len)
train_ds = ProcessesDataset(train_df_pr_sc, win_len)
val_ds = ProcessesDataset(val_df_pr_sc, win_len)

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_ds, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_ds, batch_size=batch_size, shuffle=True)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

### Modles init

In [ ]:
class NLSTM(torch.nn.Module):
    def __init__(self, input_size, hidden_layer, n_lstm, dropout=0.05):
        super(NLSTM, self).__init__()
        self.dropout = torch.nn.Dropout(dropout)
        self.lstms = torch.nn.ModuleList()
        self.n_lstm = n_lstm
        for i in range(n_lstm):
            if i == 0:
                self.lstms.append(torch.nn.LSTM(input_size, hidden_layer, 1))
            else:
                self.lstms.append(torch.nn.LSTM(hidden_layer, hidden_layer, 1))
        self.relu = torch.nn.ReLU()
        
    def forward(self, x, h=None):
        # print(x.shape)
        if self.n_lstm > 0:
            for i in range(self.n_lstm):
                if h is None:
                    x, h = self.lstms[i](x)
                else:
                    x, h = self.lstms[i](x, h)

                x = self.dropout(x)
                x = self.relu(x)
                
            return x, h
        else:
            return x, None

class Net(torch.nn.Module):
    def __init__(self, input_size, hidden_layer, n_lstm, m_lstm, n_classes, dropout=0.05):
        super(Net, self).__init__()
        
        self.bb_lstm = NLSTM(input_size, hidden_layer, n_lstm, dropout=dropout)
        nlstm_in_size = hidden_layer
        if n_lstm == 0:
            nlstm_in_size = input_size
        self.ac_lstm = NLSTM(input_size=nlstm_in_size, hidden_layer=hidden_layer,
                             n_lstm=m_lstm, dropout=dropout)
        self.te_lstm = NLSTM(input_size=nlstm_in_size, hidden_layer=hidden_layer,
                             n_lstm=m_lstm, dropout=dropout)
                    
        
        self.relu = torch.nn.ReLU()
        self.fc_ac = torch.nn.Linear(hidden_layer, n_classes)
        self.fc_te = torch.nn.Linear(hidden_layer, 1)
        
        self.n_classes = n_classes
        self.n_lstm = n_lstm
        self.m_lstm = m_lstm
        self.hidden = hidden_layer
        
        self.dropout = torch.nn.Dropout(dropout)

    def forward(self, x):
        
            # back bone lstm
        x, h = self.bb_lstm(x)
        x = self.dropout(x)
        x = self.relu(x)
            
        x_ac, _ = self.ac_lstm(x, h)
        x_te, _ = self.te_lstm(x, h)
           
        x_te = self.dropout(x_te)
        x_ac = self.dropout(x_ac)
        
        x_te = self.relu(x_te)
        x_ac = self.relu(x_ac)
        x_ac = self.fc_ac(x_ac)
        x_te = self.fc_te(x_te)
        
        x_te = self.relu(x_te)
        x_ac = torch.nn.functional.softmax(x_ac, dim=-1)
        return x_te, x_ac

In [ ]:
input_size = test_ds.tensor_data.shape[-1]

model = Net(input_size=input_size, hidden_layer=hidden_layer, n_lstm=n_lstm,
            m_lstm=m_lstm, n_classes=n_classes).to(device)

### Optimizers and losses

In [ ]:
loss_ac = torch.nn.CrossEntropyLoss()

loss_te = torch.nn.SmoothL1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
from rl4pm_lib.lstm_supervised import train_one_epoch, for_evaluate
from rl4pm_lib.utils import get_accuracy, get_f1_score, get_log_loss, get_mae_days, plot_learning

In [ ]:
train_acc = []
test_acc = []

train_mae = []
test_mae = []

test_ce = []
train_ce = []

test_f1 = []
train_f1 = []

epoches = []

for ep in range(160):
    
    train_data = train_one_epoch(dataloader=train_dataloader, device=device,
                       model=model, optimizer=optimizer,
                       loss_ac=loss_ac, loss_te=loss_te, n_classes=n_classes)
    
    eval_data = for_evaluate(val_dataloader, model, n_classes=n_classes, device=device)
    
    epoches.append(ep+1)
    
    test_acc.append(get_accuracy(true=eval_data['true_label'],
                                 pred=eval_data['pred_label']))
    train_acc.append(get_accuracy(true=train_data['true_label'],
                                  pred=train_data['pred_label']))
    
    test_mae.append(get_mae_days(true=eval_data['true_tes'],
                                 pred=eval_data['pred_tes'], scaler=scaler))
    train_mae.append(get_mae_days(true=train_data['true_tes'],
                                  pred=train_data['pred_tes'], scaler=scaler))
    
    test_ce.append(get_log_loss(true=eval_data['true_label'],
                                     pred=eval_data['pred_label']))
    train_ce.append(get_log_loss(true=train_data['true_label'],
                                  pred=train_data['pred_label']))
    
    test_f1.append(get_f1_score(true=eval_data['true_label'],
                                 pred=eval_data['pred_label']))
    train_f1.append(get_f1_score(true=train_data['true_label'],
                                  pred=train_data['pred_label']))
    
    display.clear_output(wait=True)
    plot_learning(test_acc=test_acc, train_acc=train_acc,
                  test_mae=test_mae, train_mae=train_mae,
                  test_f1=test_f1, train_f1=train_f1,
                  test_ce=test_ce, train_ce=train_ce,
                  epoches=epoches
                  )

## Hyper param tune

In [ ]:
def train_hyper(config, checkpoint_dir='hyper_tune', train_dataloader=train_dataloader, test_dataloader=test_dataloader,
                n_classes=6, n_epoches=40):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = Net(input_size=input_size, hidden_layer=config['hidden'], n_lstm=config['n_lstm'],
            m_lstm=config['m_lstm'], n_classes=n_classes).to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])
    
    loss_ac = torch.nn.CrossEntropyLoss()
    loss_te = torch.nn.SmoothL1Loss()
    for epoch in range(n_epoches):
        train_result = train_one_epoch(dataloader=train_dataloader, device=device, model=model, optimizer=optimizer,
                                       loss_ac=loss_ac, loss_te=loss_te, n_classes=n_classes)
        eval_data = for_evaluate(test_dataloader, model, n_classes=n_classes, device=device)
        
        with tune.checkpoint_dir(epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((model.state_dict(), optimizer.state_dict()), path)
            
        val_loss = get_log_loss(true=eval_data['true_label'],
                                pred=eval_data['pred_label'])
        vall_acc = get_accuracy(true=train_data['true_label'],
                                pred=train_data['pred_label'])
        tune.report(loss=val_loss, accuracy=vall_acc)

In [ ]:
from functools import partial
import os
from torch.utils.data import random_split
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
import ray
ray.shutdown()
#ray.init(log_to_driver=False)

data_dir = os.path.abspath("checkpoint")
config = {
        "n_lstm": tune.choice([1, 2, 3]),
        "m_lstm": tune.choice([1, 2, 3]),
        "lr": tune.loguniform(1e-4, 1e-1),
        "batch_size": tune.choice([128, 256]),
        "hidden": tune.choice([128, 200, 300])
    }

scheduler = ASHAScheduler(
        metric="accuracy",
        mode="max",
        max_t=700*40,
        grace_period=1,
        reduction_factor=2)

reporter = CLIReporter(metric_columns=["loss", "accuracy", "training_iteration"])

result = tune.run(partial(train_hyper, train_dataloader=train_dataloader,
                          test_dataloader=test_dataloader, n_classes=6,
                          n_epoches=40
                         ),
                  resources_per_trial={"cpu": 5, "gpu": 1},
                  num_samples=700,
                  scheduler=scheduler,
                  config=config,
                  progress_reporter=reporter)
